In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# ref: https://www.tensorflow.org/get_started/mnist/pros

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
def weight_variable(shape):
    """
    < tf.truncated_normal >
    
    The generated values follow a normal distribution 
    with specified mean and standard deviation, 
    except that values whose magnitude is more than 2 standard deviations 
    from the mean are dropped and re-picked.
    """
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    """
    x (input feature maps) : [num_batch, height, width, channel]
    W (convolution filter): [filter_height, filter_width, in_channel, out_channel]
    return (output feature maps): [num_batch, height, width, channel]
    """
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """
    x (input feature maps) : [num_batch, height, width, channel]
    ksize (window size) : [window_batch, window_height, window_width, window_channel]
    return (output feature maps) : [num_batch, height/2, width/2, channel]
    """
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [4]:
# Convolutional Neural Networks Model
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [None,28,28,1])

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)   # [None, 28, 28, 32]
h_pool1 = max_pool_2x2(h_conv1)                            # [None, 14, 14, 32]

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)   # [None, 14, 14, 64]
h_pool2 = max_pool_2x2(h_conv2)                            # [None, 7, 7, 64]

W_fc1 = weight_variable([7 * 7 * 64, 1024])              
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1) # [None, 1024]

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y = tf.matmul(h_fc1, W_fc2) + b_fc2                        # [None, 10]

In [5]:
# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cross_entropy)

In [6]:
# Test trained model
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

# Train
for i in range(5000):
    batch_xs, batch_ys = mnist.train.next_batch(50)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if i%100 == 0:
        print(sess.run(accuracy, feed_dict={x: mnist.test.images,
                                            y_: mnist.test.labels}))

0.0595
0.6017
0.7526
0.801
0.8482
0.8603
0.8815
0.8875
0.8956
0.902
0.9111
0.9105
0.9185
0.9195
0.923
0.9257
0.9282
0.9316
0.9321
0.9343
0.9349
0.9316
0.9384
0.9396
0.9375
0.9415
0.9401
0.9433
0.944
0.9441
0.9466
0.9446
0.9456
0.949
0.9483
0.9495
0.9476
0.9509
0.9506
0.9514
0.954
0.9536
0.9533
0.9531
0.9518
0.9551
0.9557
0.9562
0.9554
0.9544
